In [0]:
import tensorflow as tf

In [0]:
import numpy as np
import pandas as pd
import shutil
print(tf.__version__)

In [0]:
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
features=CSV_COLUMNS[1:len(CSV_COLUMNS)-1]
labels=CSV_COLUMNS[0]

In [0]:
df_train=pd.read_csv('./taxi-train.csv' ,header=None,names=CSV_COLUMNS)
df_valid=pd.read_csv('./taxi-valid.csv',header=None,names=CSV_COLUMNS)
df_test=pd.read_csv('./taxi-valid.csv',header=None,names=CSV_COLUMNS)

In [0]:
def make_train_input_fn(df,num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
       x=df,
       y=df[labels],
       batch_size = 128,
       shuffle = True,
       queue_capacity = 1000 
    )

In [0]:
def make_eval_input(df):
    return tf.estimator.input.pandas_input_fn(
       x=df,
       y=df[labels],
       shuffle=False,
       queue_capacity=1000,
       batch_size = 128 
    
    
    )

In [0]:
def make_prediction_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=None,
        batch_size=128,
        shuffle=False,
        queue_capacity = 1000
    
    
    )

In [0]:
#creating feture columns
def make_feature_cols():
    input_columns=[tf.feature_column.numeric_column(k) for k in features]
    return input_columns

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)
#shutil.rmtree is used to delete an entire directory
OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR , ignore_errors = True)#start fresh each time

model = tf.estimator.LinearRegressor(feature_columns=make_feature_cols , model_dir=OUTDIR)

model.train(input_fn = make_train_input_fn(df_train,num_epochs = 10))

In [0]:
def print_rmse( model , df):
    metrics = model.evaluate(input_fn = make_eval_input(df))
    print('RMSE on the dataset = {}'.format(np.sqrt(metrics['avegare_loss'])))
print_rmse(model , df_valid)    


In [0]:
predictions = model.predict(input_fn=make_prediction_input_fn(df_test))
for items in predictions:
    print(items)

In [0]:
#since the model is predicting the same value , we better try a complex model 
# a complex DNN model

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR , ignore_errors = True )
model = tf.estimator.DNNRegressor(hidden_units=[32 , 8 , 2],feature_columns = make_feature_cols() , model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train , num_epochs = 100))
print_rmse(model,df_valid)